In [81]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'IPAexGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

In [82]:
#データを整理
df23_24 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_23-24シーズン.csv')
df24_25 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_24-25シーズン.csv')
df_box = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】ボックススコア.csv')
df_game = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】試合データ.csv')
df_team = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】チームマスタ.csv')


#リーグのみ抽出
#リーグの試合IDを抽出
cupID = [500, 507]
df_gameID_cupID = df_game[df_game['カップID'].isin(cupID)]
df_gameID_cupID = df_gameID_cupID['試合ID'].unique()

#試合IDからデータを抽出
df23_24 = df23_24[df23_24['試合ID'].isin(df_gameID_cupID)]
df24_25 = df24_25[df24_25['試合ID'].isin(df_gameID_cupID)]
df_box = df_box[df_box['試合ID'].isin(df_gameID_cupID)]
df_game = df_game[df_game['試合ID'].isin(df_gameID_cupID)]


# 試合ID、チームIDごとにデータを整理
df_box_gameid = df_box.drop(columns=['ホームアウェイ','選手ID','背番号','選手名','スターティングフラグ','プレイタイム'])
df_box_gameid_sum = df_box_gameid[df_box_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()

# 試合IDごとに勝敗を割り当て
# 1. 試合IDごとに得点の最大値を取得
max_scores = df_box_gameid_sum.groupby('試合ID')['得点'].transform('max')
# 2. 自チームの得点が最大なら1（勝ち）、そうでなければ0（負け）
df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

# 試合データから失点列を作成（より効率的）
df_game_long = pd.concat([
    df_game[['試合ID', 'ホームチームID', 'アウェイ得点']].rename(
        columns={'ホームチームID': 'チームID', 'アウェイ得点': '失点'}
    ),
    df_game[['試合ID', 'アウェイチームID', 'ホーム得点']].rename(
        columns={'アウェイチームID': 'チームID', 'ホーム得点': '失点'}
    )
], ignore_index=True)

# df_box_gameid_sum に失点を結合
df_box_gameid_sum = df_box_gameid_sum.merge(
    df_game_long[['試合ID', 'チームID', '失点']],
    on=['試合ID', 'チームID'],
    how='left'
)

# 安全に成功率を計算するヘルパー
# ...existing code...
def safe_rate(df, num_col, den_col, out_col, fillna=0):
    if num_col not in df.columns or den_col not in df.columns:
        print(f"警告: 列がありません: {num_col} または {den_col}")
        return
    # 数値化（非数値は NaN）
    df[num_col] = pd.to_numeric(df[num_col], errors='coerce')
    df[den_col] = pd.to_numeric(df[den_col], errors='coerce')
    # 0 を NaN にしてゼロ除算を回避
    denom = df[den_col].replace({0: np.nan})
    # 計算（結果を float に強制）
    df[out_col] = (df[num_col] / denom).astype(float)
    # 数値化の再適用（万が一の非数を排除）
    df[out_col] = pd.to_numeric(df[out_col], errors='coerce')
    # 現実的な範囲に収める（負や 1 より大きい値が出た場合にクリップ）
    df[out_col] = df[out_col].clip(lower=0.0, upper=1.0)
    # オプションで NaN を置換
    if fillna is not None:
        df[out_col] = df[out_col].fillna(fillna)

# 成功率列を安全に作成
safe_rate(df_box_gameid_sum, '2P成功', '2P試投', '2P成功率')
safe_rate(df_box_gameid_sum, '3P成功', '3P試投', '3P成功率')
safe_rate(df_box_gameid_sum, 'フリースロー成功', 'フリースロー試投', 'フリースロー成功率')

df_box_alphas = df_box_gameid_sum[df_box_gameid_sum['チームID']== 745]

In [83]:
#目的変数を'勝敗'としたVIFを計算
df = df_box_gameid_sum.copy()

exp_val = [
    'ブロックショット','ファウル','被ブロックショット','ファストブレイクポイント',
    'アシスト','オフェンスリバウンド','ディフェンスリバウンド','スティール',
    'ターンオーバ','被ファウル','フリースロー成功率','2P成功率','3P成功率','フリースロー試投','2P試投','3P試投'
]

# 列選択（リストで）
x = df[exp_val].copy()

y = df["勝敗"]
x.corr()

vif = np.diag(np.linalg.inv(x.corr().values))
df_vif = pd.DataFrame(vif)
df_vif.columns = ["VIF"]
df_vif.index = x.corr().columns
df_vif.sort_values("VIF")

,VIF
フリースロー成功率,1.016898
ブロックショット,1.037001
ファウル,1.142821
被ブロックショット,1.190599
ディフェンスリバウンド,1.204728
スティール,1.273890
3P成功率,1.380042
ターンオーバ,1.383473
2P成功率,1.462023
ファストブレイクポイント,1.467541


In [84]:
# ===== 標準化したデータ分析 =====
# 全体チームを分析
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_box_gameid_sum[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_box_gameid_sum.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_box_gameid_sum["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display(result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.30932430559278884
            Iterations: 136
            Function evaluations: 136
            Gradient evaluations: 136


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                 2878
Model:                          Logit   Df Residuals:                     2862
Method:                           MLE   Df Model:                           15
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                  0.5600
Time:                        10:39:02   Log-Likelihood:                -877.79
converged:                       True   LL-Null:                       -1994.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const                 0        nan        nan        nan         nan         nan
ブロックショット         0.1754      0.061      2.866      0.004       0.055       0.295
ファウル            -0.5428      0.067     -8.044      0.000      -0.675      -0.411
被ブロックショット        0.0660      0.065      1.014      0.310      -0.062       0.193
ファストブレイクポイント    -0.0645      0.072     -0.897      0.370      -0.205       0.076
アシスト             0.1267      0.079      1.595      0.111      -0.029       0.282
オフェンスリバウンド       1.2105      0.093     13.077      0.000       1.029       1.392
ディフェンスリバウンド      1.7214      0.087     19.741      0.000       1.551       1.892
スティール            1.1934      0.080     14.887      0.000       1.036       1.351
ターンオーバ          -1.3412      0.082    -16.295      0.000      -1.502      -1.180
被ファウル            0.7965      0.102      7.801      0.000       0.596       0.997
フリースロー成功率        0.3685      0.062      5.966      0.000       0.247       0.490
2P成功率            1.3685      0.085     16.069      0.000       1.202       1.535
3P成功率            1.7012      0.090     18.880      0.000       1.525       1.878
フリースロー試投        -0.1831      0.105     -1.748      0.081      -0.388       0.022
2P試投            -0.7860      0.110     -7.141      0.000      -1.002      -0.570
3P試投            -0.7982      0.100     -7.999      0.000      -0.994      -0.603
================================================================================
"""

In [85]:
# ===== 標準化したデータ分析 =====
# 越谷アルファーズを分析
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_box_alphas[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_box_alphas.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_box_alphas["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display(result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.33998499525863146
            Iterations: 64
            Function evaluations: 64
            Gradient evaluations: 64


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                   60
Model:                          Logit   Df Residuals:                       48
Method:                           MLE   Df Model:                           11
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                  0.6681
Time:                        10:39:02   Log-Likelihood:                -12.434
converged:                       True   LL-Null:                       -37.460
Covariance Type:            nonrobust   LLR p-value:                 6.125e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -1.1068      0.443     -2.499      0.012      -1.975      -0.239
ブロックショット              0        nan        nan        nan         nan         nan
ファウル            -0.0544      0.474     -0.115      0.909      -0.984       0.875
被ブロックショット       -0.1649      0.541     -0.305      0.761      -1.226       0.896
ファストブレイクポイント          0        nan        nan        nan         nan         nan
アシスト                  0        nan        nan        nan         nan         nan
オフェンスリバウンド       1.1257      0.611      1.842      0.065      -0.072       2.323
ディフェンスリバウンド      1.1814      0.514      2.297      0.022       0.173       2.190
スティール            0.5761      0.503      1.146      0.252      -0.409       1.562
ターンオーバ          -0.6558      0.484     -1.354      0.176      -1.605       0.293
被ファウル            1.0017      0.501      2.000      0.046       0.020       1.984
フリースロー成功率        0.1144      0.415      0.276      0.783      -0.698       0.927
2P成功率            0.4981      0.481      1.036      0.300      -0.444       1.440
3P成功率            1.1173      0.571      1.957      0.050      -0.001       2.236
フリースロー試投              0        nan        nan        nan         nan         nan
2P試投            -0.3690      0.502     -0.735      0.463      -1.354       0.616
3P試投                  0        nan        nan        nan         nan         nan
================================================================================
"""

In [89]:
def get_cluster(df_team, df_box, names):
    ids = df_team.loc[df_team['チーム名'].isin(names), 'チームID']
    return df_box.loc[df_box['チームID'].isin(ids)].copy()

# --- cluster1全体 ---
df_cluster1 = get_cluster(df_team, df_box_gameid_sum, {
    'ファイティングイーグルス名古屋','佐賀バルーナーズ','大阪エヴェッサ',
    '広島ドラゴンフライズ','秋田ノーザンハピネッツ','茨城ロボッツ',
    '越谷アルファーズ','長崎ヴェルカ'
})

# --- cluster1（越谷以上） ---
df_cluster1_above = get_cluster(df_team, df_box_gameid_sum, {
    'ファイティングイーグルス名古屋','佐賀バルーナーズ','大阪エヴェッサ',
    '広島ドラゴンフライズ','越谷アルファーズ','長崎ヴェルカ'
})

In [ ]:
# ===== 標準化したデータ分析 =====
# 越谷アルファーズを分析
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_cluster1[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_cluster1.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_cluster1["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display('越谷アルファーズ',result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3227271089531208
            Iterations: 106
            Function evaluations: 106
            Gradient evaluations: 106


'越谷'

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                  900
Model:                          Logit   Df Residuals:                      884
Method:                           MLE   Df Model:                           15
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                  0.5477
Time:                        10:47:27   Log-Likelihood:                -278.40
converged:                       True   LL-Null:                       -615.54
Covariance Type:            nonrobust   LLR p-value:                5.621e-134
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.5351      0.108     -4.950      0.000      -0.747      -0.323
ブロックショット         0.0572      0.107      0.534      0.593      -0.153       0.267
ファウル            -0.4585      0.120     -3.827      0.000      -0.693      -0.224
被ブロックショット       -0.0146      0.118     -0.124      0.901      -0.245       0.216
ファストブレイクポイント    -0.0252      0.132     -0.191      0.849      -0.284       0.234
アシスト                  0        nan        nan        nan         nan         nan
オフェンスリバウンド       1.2483      0.166      7.540      0.000       0.924       1.573
ディフェンスリバウンド      1.6813      0.149     11.280      0.000       1.389       1.973
スティール            1.2308      0.143      8.601      0.000       0.950       1.511
ターンオーバ          -1.2060      0.146     -8.248      0.000      -1.493      -0.919
被ファウル            0.7739      0.180      4.301      0.000       0.421       1.127
フリースロー成功率        0.4084      0.107      3.827      0.000       0.199       0.618
2P成功率            1.2621      0.137      9.209      0.000       0.993       1.531
3P成功率            1.7040      0.146     11.660      0.000       1.418       1.990
フリースロー試投        -0.2638      0.181     -1.460      0.144      -0.618       0.090
2P試投            -0.6160      0.187     -3.301      0.001      -0.982      -0.250
3P試投            -0.5694      0.164     -3.479      0.001      -0.890      -0.249
================================================================================
"""

In [97]:
# ===== 標準化したデータ分析 =====
# 越谷アルファーズを分析
# 1) 標準化（DataFrameで列名保持）
scaler = StandardScaler()
X_scaled_arr = scaler.fit_transform(df_cluster1_above[exp_val])
X_scaled = pd.DataFrame(X_scaled_arr, columns=exp_val, index=df_cluster1_above.index)

# 2) 定数項を追加（DataFrameのまま）
X_scaled = sm.add_constant(X_scaled, has_constant='add')

# 3) 目的変数
y = df_cluster1_above["勝敗"]

# 4) L1正則化付きロジスティック回帰
model = sm.Logit(y, X_scaled)
result = model.fit_regularized(
    method='l1',
    alpha=1,
    maxiter=1000
)

# 5) 結果表示
display('クラスター1上位',result.summary())

# 6) 標準偏回帰係数を抽出（列名維持）
coef_df = pd.DataFrame({
    '変数': result.params.index,
    '標準偏回帰係数': result.params.values
})


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3159529700288249
            Iterations: 99
            Function evaluations: 99
            Gradient evaluations: 99


'クラスター1上位'

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     勝敗   No. Observations:                  660
Model:                          Logit   Df Residuals:                      643
Method:                           MLE   Df Model:                           16
Date:                Tue, 11 Nov 2025   Pseudo R-squ.:                  0.5675
Time:                        10:50:17   Log-Likelihood:                -196.96
converged:                       True   LL-Null:                       -455.43
Covariance Type:            nonrobust   LLR p-value:                8.866e-100
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.2979      0.126     -2.361      0.018      -0.545      -0.051
ブロックショット         0.1476      0.129      1.142      0.253      -0.106       0.401
ファウル            -0.6536      0.147     -4.442      0.000      -0.942      -0.365
被ブロックショット        0.0082      0.142      0.058      0.954      -0.270       0.286
ファストブレイクポイント  7.458e-17      0.160   4.66e-16      1.000      -0.314       0.314
アシスト            -0.1216      0.167     -0.726      0.468      -0.450       0.207
オフェンスリバウンド       1.0730      0.192      5.584      0.000       0.696       1.450
ディフェンスリバウンド      1.7220      0.179      9.617      0.000       1.371       2.073
スティール            1.1587      0.169      6.860      0.000       0.828       1.490
ターンオーバ          -1.1560      0.174     -6.644      0.000      -1.497      -0.815
被ファウル            0.6023      0.210      2.870      0.004       0.191       1.014
フリースロー成功率        0.5321      0.129      4.132      0.000       0.280       0.785
2P成功率            1.4049      0.174      8.051      0.000       1.063       1.747
3P成功率            1.9196      0.200      9.587      0.000       1.527       2.312
フリースロー試投        -0.0498      0.218     -0.229      0.819      -0.476       0.377
2P試投            -0.3490      0.238     -1.465      0.143      -0.816       0.118
3P試投            -0.3678      0.208     -1.772      0.076      -0.775       0.039
================================================================================
"""